<a href="https://colab.research.google.com/github/akds2311/Image-classification-/blob/master/aircraft_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip install -q tensorflow

In [0]:
!pip install -q keras

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from glob import glob

In [5]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16

Using TensorFlow backend.


In [0]:
IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results.

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
conv_base = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in conv_base.layers:
    layer.trainable = False

x = Flatten()(conv_base.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(7, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = conv_base.input, outputs = x)

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input, validation_split=0.1993)


In [0]:
training_dir="/content/drive/My Drive/Colab Notebooks/aircrafts"
validation_dir="/content/drive/My Drive/Colab Notebooks/aircrafts"
image_files = glob(training_dir + '/*/*.jpg')
# valid_image_files = glob(validation_dir + '/*/*.jpg')

# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)


In [14]:
training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 30, class_mode = 'categorical',subset='training')
validation_generator = training_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 10, class_mode = 'categorical', subset='validation')
training_generator.class_indices

Found 536 images belonging to 7 classes.
Found 131 images belonging to 7 classes.


{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}

In [0]:
training_images = 536
validation_images =131

In [17]:
history = model.fit_generator(training_generator,
                       steps_per_epoch = 18, 
                       epochs = 10,
                       validation_data = validation_generator,
                       validation_steps = 14) 

Epoch 1/10
18/18 [==============================] - 153s 9s/step - loss: 1.9909 - acc: 0.4222 - val_loss: 1.1873 - val_acc: 0.4714
Epoch 2/10
18/18 [==============================] - 142s 8s/step - loss: 0.9999 - acc: 0.6833 - val_loss: 0.8413 - val_acc: 0.6667
Epoch 3/10
18/18 [==============================] - 140s 8s/step - loss: 0.5054 - acc: 0.8352 - val_loss: 0.6126 - val_acc: 0.8000
Epoch 4/10
18/18 [==============================] - 141s 8s/step - loss: 0.3109 - acc: 0.9000 - val_loss: 0.6547 - val_acc: 0.7879
Epoch 5/10
18/18 [==============================] - 142s 8s/step - loss: 0.2456 - acc: 0.9333 - val_loss: 0.8096 - val_acc: 0.7143
Epoch 6/10
18/18 [==============================] - 140s 8s/step - loss: 0.2738 - acc: 0.9110 - val_loss: 0.3345 - val_acc: 0.9091
Epoch 7/10
18/18 [==============================] - 144s 8s/step - loss: 0.1093 - acc: 0.9722 - val_loss: 0.4917 - val_acc: 0.8571
Epoch 8/10
18/18 [==============================] - 141s 8s/step - loss: 0.1359 - a